# Analysis by negative sources

## Config Steps

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ViennaRNA
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 25.9 MB/s eta 0:00:00


In [3]:
import os
from Bio import SeqIO
import RNA
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from gensim.models import Word2Vec, KeyedVectors
import pickle
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
import time
import seaborn as sns
import pandas as pd
import random
from collections import Counter
from sklearn.model_selection import ParameterGrid
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

In [4]:
# Load arrays
path = "/content/drive/MyDrive/maestria/tesis"
X_train = np.load(f"{path}/X_train.npy")
X_test = np.load(f"{path}/X_test.npy")
y_train = np.load(f"{path}/y_train.npy")
y_test = np.load(f"{path}/y_test.npy")
sequence_for_screening = str(np.load(f"{path}/sequence_for_screening.npy"))
sequence_for_screening_only_negatives = str(np.load(f"{path}/sequence_for_screening_only_negatives.npy"))
chromosome_path = f"{path}/chromosome1.fa"
invalid_data_path = f"{path}/invalid_data.fa"
chr_with_mirna = f"{path}/chr1_with_mirna.fa"
neg_seq_chr1_1_path = f"{path}/negative_sequence_1.fa"

## Methods

In [14]:
def get_complementary_sequence(sequences):
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    return [''.join(complement[base] for base in seq) for seq in sequences]
def convert_RNA_sequence_to_DNA(sequence):
    return sequence.replace('U', 'T')

def convert_DNA_sequence_to_RNA(sequence):
    return sequence.replace('T', 'U')

## FastaManager

In [13]:
class FastaManager:
    FASTA = "fasta"
    FA = "fa"

    @staticmethod
    def read_fasta(file_path):
        """Read sequences from a FASTA file."""
        sequences = []
        with open(file_path, "r") as file:
            for record in SeqIO.parse(file, FastaManager.FASTA):
                sequences.append(str(record.seq))
        return sequences

    @staticmethod
    def get_all_sequences(input_dir):
        """Read FASTA files from input_dir and return sequences."""
        all_sequences = []
        for file_name in os.listdir(input_dir):
            if file_name.endswith(f".{FastaManager.FASTA}") or file_name.endswith(f".{FastaManager.FA}"):
                file_path = os.path.join(input_dir, file_name)
                sequences = FastaManager.read_fasta(file_path)
                all_sequences.extend(sequences)
        return all_sequences

    @staticmethod
    def create_fasta_dataset(input_dir, embedding):
        """Read FASTA files from input_dir and return tokenized sequences."""
        return embedding.tokenize_sequences(FastaManager.get_all_sequences(input_dir))

    @staticmethod
    def create_dataset(sequences, embedding):
        """Read list of sequences and return tokenized sequences."""
        return embedding.tokenize_sequences(sequences)

## Generate Dataset

### Positive Samples

In [5]:
df = pd.read_csv(f'{path}/dataset_DNA_chr1.csv')
df.head()

,pos_inicial,pos_final,secuencia,chromosome
0,65058434,65058508,TGCCATCCTTCAGTTATCACAGTACTGTACCTTTAGAATAGACAGC...,chr1
1,207802443,207802523,CTCCTAAAACACTGATTTCAAATGGTGCTAGATACAAAGATGGAAA...,chr1
2,109598893,109598967,GGCTGTGCCGGGTAGAGAGGGCAGTGGGAGGTAAGAGCTCTTCACC...,chr1
3,9151668,9151777,GGGCCCCACAACGTGCAGCACTTCTAGGGCAGTATACTTGCTGATT...,chr1
4,198858873,198858982,AAGCGGGGCCACAGTTGCATTCATTGTTCAGTGAGCTTGTCCACAC...,chr1


In [6]:
df = df.loc[~df['secuencia'].str.contains('N')]
df

,pos_inicial,pos_final,secuencia,chromosome
0,65058434,65058508,TGCCATCCTTCAGTTATCACAGTACTGTACCTTTAGAATAGACAGC...,chr1
1,207802443,207802523,CTCCTAAAACACTGATTTCAAATGGTGCTAGATACAAAGATGGAAA...,chr1
2,109598893,109598967,GGCTGTGCCGGGTAGAGAGGGCAGTGGGAGGTAAGAGCTCTTCACC...,chr1
3,9151668,9151777,GGGCCCCACAACGTGCAGCACTTCTAGGGCAGTATACTTGCTGATT...,chr1
4,198858873,198858982,AAGCGGGGCCACAGTTGCATTCATTGTTCAGTGAGCTTGTCCACAC...,chr1
...,...,...,...,...
500,102539644,102539726,GAGGCCTGTGTGCCTGTCAAATAGCTACAGTTAAGAAATCTTCACA...,NaN
501,26908642,26908725,TCCAACTCCCAGCTCAAGTGATCCTCCCACTTCAGCCTCTCGAGTG...,NaN
502,54878113,54878193,GCAAAAGGCGTCCTTGGGAATTCAACTGAGTACTAATCGGTAAATA...,NaN
503,47760995,47761104,CCTCCACATGCTTTCAATCCCATGGCAGCCTGTGGGTCCTGGGAAG...,NaN


In [7]:
df = df.drop_duplicates(subset='secuencia')
df

,pos_inicial,pos_final,secuencia,chromosome
0,65058434,65058508,TGCCATCCTTCAGTTATCACAGTACTGTACCTTTAGAATAGACAGC...,chr1
1,207802443,207802523,CTCCTAAAACACTGATTTCAAATGGTGCTAGATACAAAGATGGAAA...,chr1
2,109598893,109598967,GGCTGTGCCGGGTAGAGAGGGCAGTGGGAGGTAAGAGCTCTTCACC...,chr1
3,9151668,9151777,GGGCCCCACAACGTGCAGCACTTCTAGGGCAGTATACTTGCTGATT...,chr1
4,198858873,198858982,AAGCGGGGCCACAGTTGCATTCATTGTTCAGTGAGCTTGTCCACAC...,chr1
...,...,...,...,...
500,102539644,102539726,GAGGCCTGTGTGCCTGTCAAATAGCTACAGTTAAGAAATCTTCACA...,NaN
501,26908642,26908725,TCCAACTCCCAGCTCAAGTGATCCTCCCACTTCAGCCTCTCGAGTG...,NaN
502,54878113,54878193,GCAAAAGGCGTCCTTGGGAATTCAACTGAGTACTAATCGGTAAATA...,NaN
503,47760995,47761104,CCTCCACATGCTTTCAATCCCATGGCAGCCTGTGGGTCCTGGGAAG...,NaN


In [8]:
df['secuencia'] = df['secuencia'].str.replace('\n', '')
positive_sequences = df['secuencia'].to_list()

<ipython-input-8-5415f45b2c93>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['secuencia'] = df['secuencia'].str.replace('\n', '')


In [10]:
positive_sequences.extend([s[::-1] for s in positive_sequences])

In [11]:
positive_sequences.extend(get_complementary_sequence(positive_sequences))

In [12]:
len(positive_sequences)

1992

### Negative Samples

#### Ivani

In [22]:
negative_sequences_ivani = []
negative_sequences_ivani = FastaManager.read_fasta(f"{path}/pseudo_hairpin_human.fasta")[:1992]
negative_sequences_ivani = [convert_RNA_sequence_to_DNA(seq) for seq in negative_sequences_ivani]
len(negative_sequences_ivani)

1992

#### Ensemble

In [23]:
negative_sequences_ensemble = []
negative_sequence_ensemble = FastaManager.read_fasta(neg_seq_chr1_1_path)[0][:10000]
substring_length = 100
substrings = []
for idx in range(0, len(negative_sequence_ensemble)-substring_length, int(substring_length / 19)):
        new_sequence = negative_sequence_ensemble[idx:idx+substring_length]
        substrings.append(new_sequence)
negative_sequences_ensemble.extend(substrings)
len(negative_sequences_ensemble)

1980

#### Hybrid

In [26]:
negative_sequences_hybrid = []
negative_sequences_hybrid.extend(negative_sequences_ivani[:len(negative_sequences_ivani) // 2])
negative_sequences_hybrid.extend(negative_sequences_ensemble[:len(negative_sequences_ensemble) // 2])
len(negative_sequences_hybrid)

1986

## MiRNA2Vec

In [36]:
class MiRNA2Vec:
    def __init__(self, k_mers: int = 3, vector_size: int = 100, epochs: int = 5):
        self.__model = None
        self.k_mers = k_mers
        self.vector_size = vector_size
        self.epochs = epochs
        self.window = 5
        self.min_count = 1
        self.workers = 4
        self.skip_gram = 1

    @property
    def model(self):
        return self.__model

    def __get_tokens(self, sequence):
        return [sequence[i:i + self.k_mers] for i in range(len(sequence) - self.k_mers + 1)]

    def tokenize_sequences(self, sequences):
        """Tokenize sequences into k-mers."""
        tokenized_sequences = []
        for seq in sequences:
            tokens = self.__get_tokens(seq)
            tokenized_sequences.append(tokens)
        return tokenized_sequences

    def train_word2vec(self, tokenized_sequences):
        """Train Word2Vec model on tokenized sequences."""
        self.__model = Word2Vec(sentences=tokenized_sequences,
                                vector_size=self.vector_size,
                                window=self.window,
                                min_count=self.min_count,
                                workers=self.workers,
                                epochs=self.epochs,
                                sg=self.skip_gram)
        return self.__model

    def get_average_embeddings(self, sequences):
        """Get average embedding for each sequence."""
        embeddings = []
        for seq in sequences:
            tokens = self.__get_tokens(seq)
            token_embeddings = [self.model.wv[token] for token in tokens if token in self.model.wv]
            if token_embeddings:
                avg_embedding = np.mean(token_embeddings, axis=0)
            else:
                avg_embedding = np.zeros(self.model.vector_size)
            embeddings.append(avg_embedding.tolist())
        return np.array(embeddings)

    def load_model(self, path, tokenized_sequences):
        self.__model = Word2Vec(sentences=tokenized_sequences,
                                vector_size=self.vector_size,
                                window=self.window,
                                min_count=self.min_count,
                                workers=self.workers,
                                epochs=self.epochs)
        self.__model.wv = KeyedVectors.load_word2vec_format(path,
                                          binary=False)  # Change binary to False if the model is in text format
        # Continue training the Word2Vec model with your new sequences
        self.__model.build_vocab(tokenized_sequences, update=True)  # Update the vocabulary with new sentences
        self.__model.train(tokenized_sequences, total_examples=self.__model.corpus_count, epochs=self.__model.epochs)
        return self.__model


## Test with RF

### Ivani

In [34]:
y_positive = np.ones(len(positive_sequences))
y_negative = np.zeros(len(negative_sequences_ivani))

X = positive_sequences + negative_sequences_ivani
y = np.append(y_positive, y_negative)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=44)

In [37]:
# Create miRNA2Vec instance
miRNA2Vec = MiRNA2Vec(k_mers=5, vector_size=32, epochs=3)
# Create the dataset for Word2Vec
tokenized_sequences = FastaManager.create_dataset(X_train, miRNA2Vec)

# Train the Word2Vec model
miRNA2Vec.train_word2vec(tokenized_sequences)
# Get embeddings for sequences
X_train_mirna2vec = miRNA2Vec.get_average_embeddings(X_train)
X_test_mirna2vec = miRNA2Vec.get_average_embeddings(X_test)
# Train RF
rf_model = RandomForestClassifier(
        n_estimators=200,
        max_depth=20,
        min_samples_split=10,
        class_weight=None,
        random_state=42
    )
rf_model.fit(X_train_mirna2vec, y_train)
# Predict and evaluate
y_pred = rf_model.predict(X_test_mirna2vec)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1: {f1 * 100:.2f}%")

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 83.56%
Precision: 88.47%
Recall: 77.14%
F1: 82.42%
Confusion Matrix:
[[359  40]
 [ 91 307]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.80      0.90      0.85       399
         1.0       0.88      0.77      0.82       398

    accuracy                           0.84       797
   macro avg       0.84      0.84      0.83       797
weighted avg       0.84      0.84      0.83       797



### Ensemble

In [38]:
y_positive = np.ones(len(positive_sequences))
y_negative = np.zeros(len(negative_sequences_ensemble))

X = positive_sequences + negative_sequences_ensemble
y = np.append(y_positive, y_negative)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=44)

In [39]:
# Create miRNA2Vec instance
miRNA2Vec = MiRNA2Vec(k_mers=5, vector_size=32, epochs=3)
# Create the dataset for Word2Vec
tokenized_sequences = FastaManager.create_dataset(X_train, miRNA2Vec)

# Train the Word2Vec model
miRNA2Vec.train_word2vec(tokenized_sequences)
# Get embeddings for sequences
X_train_mirna2vec = miRNA2Vec.get_average_embeddings(X_train)
X_test_mirna2vec = miRNA2Vec.get_average_embeddings(X_test)
# Train RF
rf_model = RandomForestClassifier(
        n_estimators=200,
        max_depth=20,
        min_samples_split=10,
        class_weight=None,
        random_state=42
    )
rf_model.fit(X_train_mirna2vec, y_train)
# Predict and evaluate
y_pred = rf_model.predict(X_test_mirna2vec)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1: {f1 * 100:.2f}%")

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 91.07%
Precision: 97.13%
Recall: 84.71%
F1: 90.50%
Confusion Matrix:
[[386  10]
 [ 61 338]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.86      0.97      0.92       396
         1.0       0.97      0.85      0.90       399

    accuracy                           0.91       795
   macro avg       0.92      0.91      0.91       795
weighted avg       0.92      0.91      0.91       795



### Hybrid

In [40]:
y_positive = np.ones(len(positive_sequences))
y_negative = np.zeros(len(negative_sequences_hybrid))

X = positive_sequences + negative_sequences_hybrid
y = np.append(y_positive, y_negative)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=44)

In [41]:
# Create miRNA2Vec instance
miRNA2Vec = MiRNA2Vec(k_mers=5, vector_size=32, epochs=3)
# Create the dataset for Word2Vec
tokenized_sequences = FastaManager.create_dataset(X_train, miRNA2Vec)

# Train the Word2Vec model
miRNA2Vec.train_word2vec(tokenized_sequences)
# Get embeddings for sequences
X_train_mirna2vec = miRNA2Vec.get_average_embeddings(X_train)
X_test_mirna2vec = miRNA2Vec.get_average_embeddings(X_test)
# Train RF
rf_model = RandomForestClassifier(
        n_estimators=200,
        max_depth=20,
        min_samples_split=10,
        class_weight=None,
        random_state=42
    )
rf_model.fit(X_train_mirna2vec, y_train)
# Predict and evaluate
y_pred = rf_model.predict(X_test_mirna2vec)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1: {f1 * 100:.2f}%")

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 82.66%
Precision: 86.88%
Recall: 76.21%
F1: 81.20%
Confusion Matrix:
[[360  45]
 [ 93 298]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.79      0.89      0.84       405
         1.0       0.87      0.76      0.81       391

    accuracy                           0.83       796
   macro avg       0.83      0.83      0.83       796
weighted avg       0.83      0.83      0.83       796

